# Klassifikation mit einem Neuronalen Netz

## Trainingsdaten

In [2]:
# %pip install torch
# %pip install torchvision
# %pip install matplotlib

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.



In [14]:
from torchvision import datasets, transforms
from torch.utils.data import TensorDataset, DataLoader
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np

def load(data):
    return np.load(data)['arr_0']
# load dataset
train_data = load('Data\K49-data\k49-train-imgs.npz')
test_data = load('Data\K49-data\k49-test-imgs.npz')

train_labels = load('Data\K49-data\k49-train-labels.npz')
test_labels = load('Data\K49-data\k49-test-labels.npz')

In [4]:
train_data.data.shape, test_data.data.shape

((232365, 28, 28), (38547, 28, 28))

## Definition des Neuronalen Netzes

In [16]:
# set parameters for feed forward neural network
input_size = 28*28 #=784
hidden_size = 240100
num_classes = 49
num_epochs = 10
batch_size = 500
learning_rate = 0.001

#combine labels and data
train_data = train_data.astype(np.float32)  # Or any other appropriate data type

dataset = TensorDataset(torch.tensor(train_data), torch.tensor(train_labels))

# create dataloader

train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

# create neural network
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.l1 = nn.Linear(input_size, hidden_size).to(dtype=torch.float32) # Hidden layer
        self.relu = nn.ReLU() # Activation function
        self.l2 = nn.Linear(hidden_size, num_classes).to(dtype=torch.float32) # Output layer

    def forward(self, x):  # Forward pass
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out)
        return out

model = NeuralNet(input_size, hidden_size, num_classes)


Wie erwartet sind die Gewichte des Neuronalen Netzes vor dem Training zufällig gewählt, so dass die Klassifikation misslingt.

## Training des Neuronalen Netz

In [17]:

# loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (train_data, train_labels) in enumerate(train_loader):
        # reshape images to (batch_size, input_size)
        train_data = train_data.reshape(-1, 28*28)

        # forward pass
        outputs = model(train_data)
        loss = criterion(outputs, train_labels)

        # backward and optimize
        optimizer.zero_grad() # clear gradients
        loss.backward() # backpropagation
        optimizer.step() # update weights

        if (i+1) % 100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'
                  .format(epoch+1, num_epochs, i+1, total_step, loss.item()))



Epoch [1/10], Step [100/465], Loss: 0.9882
Epoch [1/10], Step [200/465], Loss: 0.7625
Epoch [1/10], Step [300/465], Loss: 0.8917
Epoch [1/10], Step [400/465], Loss: 0.8186
Epoch [2/10], Step [100/465], Loss: 0.5892
Epoch [2/10], Step [200/465], Loss: 0.7649
Epoch [2/10], Step [300/465], Loss: 0.7452
Epoch [2/10], Step [400/465], Loss: 0.8353
Epoch [3/10], Step [100/465], Loss: 0.6449
Epoch [3/10], Step [200/465], Loss: 0.5245
Epoch [3/10], Step [300/465], Loss: 0.7068
Epoch [3/10], Step [400/465], Loss: 0.9339
Epoch [4/10], Step [100/465], Loss: 0.7948
Epoch [4/10], Step [200/465], Loss: 0.5628
Epoch [4/10], Step [300/465], Loss: 0.6344
Epoch [4/10], Step [400/465], Loss: 0.8943
Epoch [5/10], Step [100/465], Loss: 0.5903
Epoch [5/10], Step [200/465], Loss: 0.4670
Epoch [5/10], Step [300/465], Loss: 0.5116
Epoch [5/10], Step [400/465], Loss: 0.7290
Epoch [6/10], Step [100/465], Loss: 0.5113
Epoch [6/10], Step [200/465], Loss: 0.5959
Epoch [6/10], Step [300/465], Loss: 0.7190
Epoch [6/10

## Test des Neuronalen Netzes

In [18]:
# test the model
# In test phase, we don't need to compute gradients (for memory efficiency)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, 28*28)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy of the model on test images: {} %'.format(100 * correct / total))


KeyboardInterrupt: 

hidden_size = 240100

num_classes = 49

num_epochs = 10

batch_size = 500

learning_rate = 0.001

Accuracy = 

## Test mit kleinerem Trainingsdatensatz

In [13]:
# choose a small traings set randomly
total_images = len(datasets.MNIST(root='data', train=True, download=True))  # Gesamtanzahl der Trainingsbilder
train_size = 700 #int(0.7 * total_images)  # 70% der Bilder für das Training

train_data_small = datasets.MNIST(root='data', train=True, download=True, transform=transforms.ToTensor())
train_data_small.data = train_data.data[:train_size]
train_data_small.targets = train_data.targets[:train_size]

test_data_small = datasets.MNIST(root='data', train=True, download=True, transform=transforms.ToTensor())
test_data_small.data = test_data.data[train_size:]
test_data_small.targets = test_data.targets[train_size:]

100.0%


Extracting data\MNIST\raw\train-images-idx3-ubyte.gz to data\MNIST\raw


100.0%


Extracting data\MNIST\raw\train-labels-idx1-ubyte.gz to data\MNIST\raw




33.8%

100.0%
100.0%


Extracting data\MNIST\raw\t10k-images-idx3-ubyte.gz to data\MNIST\raw

Extracting data\MNIST\raw\t10k-labels-idx1-ubyte.gz to data\MNIST\raw



AttributeError: 'Tensor' object has no attribute 'targets'

In [ ]:
train_data, train_data_small

(Dataset MNIST
     Number of datapoints: 60000
     Root location: data
     Split: Train
     StandardTransform
 Transform: ToTensor(),
 Dataset MNIST
     Number of datapoints: 42000
     Root location: data
     Split: Train
     StandardTransform
 Transform: ToTensor())

In [ ]:
# train model_small on train_data_small
# print loss and accuracy on train_data_small and test_data_small

batch_size = 10
num_epochs = 10
learning_rate = 0.001
report = 5

model_small = NeuralNet(input_size, hidden_size, num_classes)



# create dataloader
train_loader_small = DataLoader(dataset=train_data_small, batch_size=batch_size, shuffle=True)
test_loader_small = DataLoader(dataset=test_data_small, batch_size=batch_size, shuffle=False)


# loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_small.parameters(), lr=learning_rate)

# train the model
total_step = len(train_loader_small)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader_small):
        # reshape images to (batch_size, input_size)
        images = images.reshape(-1, 28*28)

        # forward pass
        outputs = model_small(images)
        loss = criterion(outputs, labels)

        # backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # print loss on train_data_small and test_data_small
        if (i+1) % report == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'
                  .format(epoch+1, num_epochs, i+1, total_step, loss.item()))
        # print accuracy on train_data_small and test_data_small
        if (i+1) % report == 0:
            # In test phase, we don't need to compute gradients (for memory efficiency)
            with torch.no_grad():
                correct = 0
                total = 0
                for images, labels in train_loader_small:
                    images = images.reshape(-1, 28*28)
                    outputs = model_small(images)
                    _, predicted = torch.max(outputs.data, 1)
                    total += labels.size(0)
                    correct += (predicted == labels).sum().item()

                print('Train Accuracy of the model on the train images: {} %'.format(100 * correct / total))

            # In test phase, we don't need to compute gradients (for memory efficiency)
            with torch.no_grad():
                correct = 0
                total = 0
                for images, labels in test_loader_small:
                    images = images.reshape(-1, 28*28)
                    outputs = model_small(images)
                    _, predicted = torch.max(outputs.data, 1)
                    total += labels.size(0)
                    correct += (predicted == labels).sum().item()

                print('Test Accuracy of the model on the test images: {} %'.format(100 * correct / total))

# test the model
# In test phase, we don't need to compute gradients (for memory efficiency)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader_small:
        images = images.reshape(-1, 28*28)
        outputs = model_small(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy of the model on the test images: {} %'.format(100 * correct / total))



Epoch [1/10], Step [5/100], Loss: 2.1572
Train Accuracy of the model on the train images: 21.4 %
Test Accuracy of the model on the test images: 23.244444444444444 %
Epoch [1/10], Step [10/100], Loss: 2.0135
Train Accuracy of the model on the train images: 35.4 %
Test Accuracy of the model on the test images: 34.65555555555556 %
Epoch [1/10], Step [15/100], Loss: 1.8571
Train Accuracy of the model on the train images: 63.5 %
Test Accuracy of the model on the test images: 60.94444444444444 %
Epoch [1/10], Step [20/100], Loss: 1.7955
Train Accuracy of the model on the train images: 70.7 %
Test Accuracy of the model on the test images: 67.18888888888888 %
Epoch [1/10], Step [25/100], Loss: 1.1323
Train Accuracy of the model on the train images: 72.6 %
Test Accuracy of the model on the test images: 68.07777777777778 %
Epoch [1/10], Step [30/100], Loss: 0.8220
Train Accuracy of the model on the train images: 73.7 %
Test Accuracy of the model on the test images: 68.26666666666667 %
Epoch [1/1

In [ ]:
# test the model
# In test phase, we don't need to compute gradients (for memory efficiency)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, 28*28)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))


Test Accuracy of the model on the 10000 test images: 96.85 %
